# Desafio

### Acesse a api da camara, e recupere os reembolsos do ano de 2018

In [1]:
import requests

# fazendo o download do conteúdo do arquivo
url = "http://www.camara.leg.br/cotas/Ano-2018.csv.zip"
r = requests.get(url)  # pego o conteudo 

In [7]:
r.content
r.status_code

200

In [8]:
with open("Ano-2018.csv.zip", "wb") as arquivo:
    arquivo.write(r.content)


In [9]:
from zipfile import ZipFile

zipado = ZipFile("Ano-2018.csv.zip", "r")
zipado.namelist()

['Ano-2018.csv']

In [11]:
zipado.extract(member="Ano-2018.csv", path="reembolso-2018")

'reembolso-2018\\Ano-2018.csv'

In [17]:
zipado.close()

### Leia o Arquivo csv 

In [26]:
import pandas as pd
pd.set_option('display.max_columns', None)
DTYPE = {
    'txNomeParlamentar': str,
    'ideCadastro': str,
    'nuCarteiraParlamentar': str,
    'nuLegislatura': str,
    'sgUF': str,
    'sgPartido': str,
    'codLegislatura': str,
    'numSubCota': str,
    'txtDescricao': str,
    'numEspecificacaoSubCota': str,
    'txtDescricaoEspecificacao': str,
    'txtFornecedor': str,
    'txtCNPJCPF': str,
    'txtNumero': str,
    'indTipoDocumento': str,
    'datEmissao': str,
    'vlrDocumento': float,
    'vlrGlosa': str,
    'vlrLiquido': float,
    'numMes': str,
    'numAno': str,
    'numParcela': str,
    'txtPassageiro': str,
    'txtTrecho': str,
    'numLote': str,
    'numRessarcimento': str,
    'nuDeputadoId': str,
    'ideDocumento': str,
}

df_ano_2018 = pd.read_csv("reembolso-2018/Ano-2018.csv", sep=';', dtype=DTYPE)
df_ano_2018.shape

(292632, 31)

### Visualize 10 linhas aleatórias

In [27]:
df_ano_2018.sample(10)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
99721,DÉCIO LIMA,3.885824e+10,141413,475,2015,SC,PT,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,VIA SUL LTDA,045.423.370/0018-1,644751,0,2018-12-10T00:00:00,146.28,0,146.28,12,2018,0,NaN,NaN,1550176,NaN,NaN,1810,6727252,https://www.camara.leg.br/cota-parlamentar/doc...
34223,Daniel Almeida,7.894091e+09,74060,188,2015,BA,PCdoB,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - GOL,075.756.510/0015-9,Bilhete: PJHSKZ,0,2018-04-11T00:00:00,733.93,0,733.93,4,2018,0,Daniel Almeida,BSB/SSA,0,0,NaN,1562,1553473,NaN
26474,Ricardo Barros,4.247898e+10,73788,462,2015,PR,PP,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - TAM,020.128.620/0016-0,Bilhete: 957-2181.347298,0,2018-10-19T00:00:00,610.39,0,610.39,10,2018,0,Ricardo Barros,BSB/CWB,0,0,NaN,1155,1608187,NaN
51476,JOSUÉ BENGTSON,9.673505e+09,74351,30,2015,PA,PTB,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - GOL,075.756.510/0015-9,Bilhete: OM7WKQ,0,2018-04-17T00:00:00,774.17,0,774.17,4,2018,0,JOSUÉ BENGTSON,BEL/BSB,0,0,NaN,1281,1554840,NaN
256895,Soraya Santos,4.912529e+10,178946,328,2015,RJ,PR,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - TAM,020.128.620/0016-0,Comp-957-2186.485922,0,2018-12-17T00:00:00,-1662.17,0,-1662.17,12,2018,0,Soraya Santos,BSB/SDU,0,0,NaN,2922,1635251,NaN
103308,Fábio Faria,2.128728e+09,141428,119,2015,RN,PSD,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - TAM,020.128.620/0016-0,Bilhete: 957-2168.101434,0,2018-05-23T00:00:00,241.03,0,241.03,5,2018,0,ANTONIO BEZERRA,BSB/NAT,0,0,NaN,1829,1582003,NaN
236585,BETO SALAME,4.110865e+10,178906,20,2015,PA,PP,55,122,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",0,NaN,SINTAXI/DF,074.241.090/0010-3,S/N,1,2018-04-03T00:00:00,20.00,0,20.00,4,2018,0,NaN,NaN,1483901,NaN,NaN,3029,6551397,https://www.camara.leg.br/cota-parlamentar/doc...
246673,Aliel Machado,6.908053e+09,178927,441,2015,PR,PSB,55,999,PASSAGEM AÉREA - RPA,0,NaN,Cia Aérea - TAM,020.128.620/0016-0,Bilhete: 957-2187.190347,0,2018-12-28T00:00:00,1443.39,0,1443.39,12,2018,0,Aliel Machado,BSB/CWB,0,0,NaN,2915,1630101,NaN
252208,Toninho Wandscheer,1.859104e+10,178934,470,2015,PR,PROS,55,10,TELEFONIA,0,NaN,RAMAL,000.000.000/0000-6,6497723,0,NaN,38.41,0,38.41,1,2018,0,NaN,NaN,0,0,NaN,2940,0,NaN
147352,Danrlei de Deus Hinterholz,7.015732e+10,160552,495,2015,RS,PSD,55,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,GARAGEM GETULIO VARGAS LTDA,945.838.120/0015-0,42424,4,2018-08-15T00:00:00,203.36,0,203.36,8,2018,0,NaN,NaN,1531747,NaN,NaN,2242,6677071,http://camara.leg.br/cota-parlamentar/nota-fis...


### Existem registros com `ideDocumento` nulo?

### Converta a coluna `datEmissao` para `datetime`, e descubra quantos reembolsos aconteram no primeiro semestre

### Limpe a coluna `txtCNPJCPF`: ela deve conter somente dígitos numéricos

In [ ]:
# vamos trocar qualquer caracter que não seja um dígito número por '', essa operação remove os dígitos
df_reimbursements["txtCNPJCPF"] = df_reimbursements...

### Quais são os valores únicos do campo `indTipoDocumento`? Substitua o conteúdo pela respectiva legenda

*Dica*: 0 (Zero), para Nota Fiscal; 1 (um), para Recibo; e 2, para Despesa no Exterior.